In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader

In [2]:
import json
import urllib

url = "https://raw.githubusercontent.com/Chaoyuuu/Gather-Town-Datasets/master/datasets-beautiful-rooms.json"
response = urllib.request.urlopen(url)
data = json.loads(response.read())
data_length = len(data)
# input_data = torch.zeros((data_length, 17, 13, 10))



count = 0
for i in range(data_length):
    room_id = data[i]["id"]
    if "Han"not in room_id and "Chao"not in room_id and "Nong"not in room_id: 
        count += 1
#         print(room_id)
        
print("count: " + str(count))
input_data = torch.zeros((count, 17, 13, 10))

"""
i = -1
for d in range(data_length):
    room_id = data[i]["id"]
    if "Chao" in room_id: 
        i += 1
        room = data[d]["room"]
        for k in range(len(room)):
            object = room[k]
            name = object["_name"]
            if name in room_decoration_label_dict:
                label = room_decoration_label_dict[name]
                x = object["x"]
                y = object["y"]
                orientation = object["orientation"]

                room_decoration_data[i][label][y][x] = 1
                if orientation == 0:
                    room_decoration_data[i][encode_size - 1][y][x] = -1
                elif orientation == 1:
                    room_decoration_data[i][encode_size - 2][y][x] = -1
                elif orientation == 2:
                    room_decoration_data[i][encode_size - 1][y][x] = 1
                else:
                    room_decoration_data[i][encode_size - 2][y][x] = 1
"""

n = -1
for i in range(data_length):
    room_id = data[i]["id"]
    if "Han"not in room_id and "Chao"not in room_id and "Nong"not in room_id: 
        n += 1
        for j in range(len(data[i]["room"])):
            input_data[n][data[i]["room"][j]["label"]][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1
            if data[i]["room"][j]["orientation"] == 0:
                input_data[n][16][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = -1
            elif data[i]["room"][j]["orientation"] == 1:
                input_data[n][15][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = -1
            elif data[i]["room"][j]["orientation"] == 2:
                input_data[n][16][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1
            else:
                input_data[n][15][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1



z_dim = 4
x_dim = 17*13*10
bs = count
epoch_round = 20
print("datasets number = " + str(count))

torch.set_printoptions(profile="full")    
# print(input_data[0])
train_loader = DataLoader(dataset=input_data, batch_size=bs, shuffle=True)
# train_loader = DataLoader(dataset=room_decoration_data, batch_size=bs, shuffle=True)

count: 120
datasets number = 120


In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, x_dim))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=x_dim, h_dim1= 512, h_dim2=256, z_dim=z_dim)
if torch.cuda.is_available():
    vae.cuda()

In [4]:
vae

VAE(
  (fc1): Linear(in_features=2210, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=4, bias=True)
  (fc32): Linear(in_features=256, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=2210, bias=True)
)

In [5]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, x_dim), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [6]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        print(list(data.size())) # [81, 17, 13, 10]
        
        recon_batch, mu, log_var = vae(data) #torch.float32
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [7]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [8]:
for epoch in range(1, epoch_round):
    train(epoch)
#     test()

[120, 17, 13, 10]
Train Epoch: 1 [0/120 (0%)]	Loss: 1533.597135
====> Epoch: 1 Average loss: 1533.5971
[120, 17, 13, 10]
Train Epoch: 2 [0/120 (0%)]	Loss: 1468.595703
====> Epoch: 2 Average loss: 1468.5957
[120, 17, 13, 10]
Train Epoch: 3 [0/120 (0%)]	Loss: 1393.290755
====> Epoch: 3 Average loss: 1393.2908
[120, 17, 13, 10]
Train Epoch: 4 [0/120 (0%)]	Loss: 1283.858203
====> Epoch: 4 Average loss: 1283.8582
[120, 17, 13, 10]
Train Epoch: 5 [0/120 (0%)]	Loss: 1146.532682
====> Epoch: 5 Average loss: 1146.5327
[120, 17, 13, 10]
Train Epoch: 6 [0/120 (0%)]	Loss: 950.180859
====> Epoch: 6 Average loss: 950.1809
[120, 17, 13, 10]
Train Epoch: 7 [0/120 (0%)]	Loss: 772.908919
====> Epoch: 7 Average loss: 772.9089
[120, 17, 13, 10]
Train Epoch: 8 [0/120 (0%)]	Loss: 564.056706
====> Epoch: 8 Average loss: 564.0567
[120, 17, 13, 10]
Train Epoch: 9 [0/120 (0%)]	Loss: 352.511556
====> Epoch: 9 Average loss: 352.5116
[120, 17, 13, 10]
Train Epoch: 10 [0/120 (0%)]	Loss: 195.347917
====> Epoch: 10 A

In [49]:
with torch.no_grad():
    z = torch.randn(1, z_dim).cuda()
    sample = vae.decoder(z).cuda()
    
    torch.set_printoptions(profile="full")    
    data = sample[0]
    
    output_tensor = torch.zeros((17, 13, 10))
    threshold = 0.28
    
    # reshape sample(1*200) -> output_tensor(17, 13, 10)
    for k, batch in enumerate(sample):
        i = 0
        for depth in range(17):
            for col in range(13):
                for row in range(10):
                    #print("depth, col, row: " + str(row)+", " + str(col) +", ", str(depth))
                    data = batch[i]
                    i += 1
                    if data >= threshold:
                        output_tensor[depth][col][row] = data
                        
        
    # print room GUI
    object_dict = {
        0: 'WB',
        1: 'PR',
        2: 'T2',
        3: 'TV',
        4: 'B4',
        5: 'PP',
        6: 'MB',
        7: 'CC',
        8: 'CS',
        9: 'T3',
        10: 'B2',
        11: 'LP',
        12: 'MP',
        13: 'LB',
        14: 'DC'
    }
    
    object_name_dict = {
        0:  'Whiteboard',
        1:  'Projector Screen',
        2:  'Chippendale Table (2x3)',
        3:  'TV (Flatscreen)',
        4:  'Bookshelf (2x4)',
        5:  'Potted Plant (Spikey)',
        6:  'Mod Chair',
        7:  'Captain’s Chair',
        8:  'Chair (Simple)',
        9:  'Chippendale Table (3x3)',
        10: 'Bookshelf [Tall] (1x2)',
        11: 'Laptop',
        12: 'Microphone',
        13: 'Lucky Bamboo',
        14: 'Dining Chair (Square)'
    }


    room = [['--']*10 for i in range(13)]
    for col in range(13):
        for row in range(10):
            object_duplicate_flage = 0
            max_value = 0
            max_index = -1
            for depth in range(15):
                data = output_tensor[depth][col][row]
                
                if data >= threshold and max_value < data:
                    max_index = depth
                    
            if max_index != -1:
                room[col][row] = object_dict[max_index]
    
    for col in range(13):
        print(room[col])
        
        
#     print(sample)
    
    
    # convert to json
    room_json = {
        "generator": "vae",
        "room": []
    }
    
    for col in range(13):
        for row in range(10):
            max_value = 0
            max_index = -1
            for depth in range(15):
                data = output_tensor[depth][col][row]

                if data >= threshold and max_value < data:
                    max_index = depth

            if max_index != -1:
                object = {
                    "x": col,
                    "y": row,
                    "name": object_name_dict[max_index],
                    "orientation": 0
                }
                room_json["room"].append(object)
                
#     print(room_json)
                    
    # print room with object
    # WB: Whiteboard
    # PR: Projector Screen
    # T2: Chippendale Table (2x3)
    # TV: TV (Flatscreen)
    # B4: Bookshelf (2x4)
    # PP: Potted Plant (Spikey)
    # MB: Mod Chair
    # CC: Captain’s Chair
    # CS: Chair (Simple)
    # T3: Chippendale Table (3x3)
    # B2: Bookshelf [Tall] (1x2)
    # LP: Laptop
    # MP: Microphone
    # LB: Lucky Bamboo
    # DC: Dining Chair (Square)

['LB', 'PP', '--', '--', 'TV', '--', 'B4', '--', 'PP', 'LB']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', 'WB', '--', '--', '--', '--', '--']
['--', '--', '--', '--', 'T2', '--', 'CS', '--', '--', '--']
['--', '--', '--', 'CS', '--', '--', 'DC', '--', '--', '--']
['--', '--', 'T3', 'LP', '--', 'T3', 'LP', '--', '--', '--']
['--', 'CS', 'DC', 'CS', '--', '--', 'CS', 'CS', 'CS', '--']
['--', '--', '--', 'LP', '--', '--', 'DC', '--', '--', '--']
['--', '--', '--', 'DC', '--', '--', 'DC', '--', '--', '--']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']


In [43]:
# PATH = "./outputModel/vae-johnny"

# # Save
# torch.save({
#     'model': vae,
#     'z_dim': z_dim,
#     'threshold': threshold
#     }, PATH)